In [47]:
## Import modules 

import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

## read data 
CIP_data = pd.read_csv("CIP_Resistant_disagregated.csv")

print(CIP_data.columns)

Index(['Unnamed: 0', 'CLINIC', 'YEAR', 'GENDERSP', 'Susceptible'], dtype='object')


In [ ]:
## Multivariable model with X1 = CLINIC and X2 = GENDERSP
X1 = CIP_data[["CLINIC"]]
X2 = CIP_data[["GENDERSP"]]
y = CIP_data[["Susceptible"]]


## OneHot encoding - CLINIC
encoder_loc = OneHotEncoder(drop='first') #drop='first' handles collinearity https://geoffruddock.com/one-hot-encoding-plus-linear-regression-equals-multi-collinearity/ BUT NOT WORKING 
encoder_categories_loc  = encoder_loc.fit(CIP_data[['CLINIC']]).categories_
#print(encoder_categories_loc)
encoder_categories_loc  = encoder_categories_loc[0].tolist()
#print(len(encoder_categories_loc))
encoded_loc = encoder_loc.fit_transform(CIP_data[['CLINIC']])
encoder_df_loc  = pd.DataFrame(encoder_loc.fit_transform(CIP_data[['CLINIC']]).toarray())
CIP_data_encoded_loc = CIP_data.join(encoder_df_loc)
#print(CIP_data_encoded_loc.shape)
CIP_data_encoded_loc.head(12)
col_names = list(CIP_data.columns) + encoder_categories_loc[1:] #for some reason when i get the categories it doesn't drop the first?
print(col_names)
CIP_data_encoded_loc.columns = col_names




In [ ]:
## OneHot encoding - GENDERSP
encoder_gen = OneHotEncoder(drop='first')
encoder_categories_gend  = encoder_gen.fit(CIP_data_encoded_loc[['GENDERSP']]).categories_
encoder_categories_gend  = encoder_categories_gend[0].tolist()
encoder_df_gend  = pd.DataFrame(encoder_gen.fit_transform(CIP_data_encoded_loc[['GENDERSP']]).toarray())
CIP_data_encoded_gend = CIP_data_encoded_loc.join(encoder_df_gend)
CIP_data_encoded_gend.head(12)
col_names = list(CIP_data_encoded_loc.columns) + encoder_categories_gend[1:]
#print(col_names)
CIP_data_encoded_gend.columns = col_names
#
print(len(list(CIP_data["GENDERSP"].unique())))
print(len(list(CIP_data["CLINIC"].unique())))


CIP_data_encoded_gend.head(2)




In [ ]:
## save file

CIP_data_encoded = CIP_data_encoded_gend ## reassign for sake of straighforwardness 

CIP_data_encoded.to_csv("CIP_data_encoded.csv")

In [4]:
############### Need to redo encoding. No longer want each location to be an indiviudal 
############### factor; group into broader locations 
############### Also, add in (a) prevalence in region in previous year and (b) prevalence across clinic in previous year 
CIP_data = pd.read_csv("CIP_Resistant_disagregated.csv")

encoder_gen = OneHotEncoder(drop='first')
encoder_categories_gend  = encoder_gen.fit(CIP_data[['GENDERSP']]).categories_
encoder_categories_gend  = encoder_categories_gend[0].tolist()
encoder_df_gend  = pd.DataFrame(encoder_gen.fit_transform(CIP_data[['GENDERSP']]).toarray())
CIP_data_encoded_gend = CIP_data.join(encoder_df_gend)
CIP_data_encoded_gend.head(12)
col_names = list(CIP_data.columns) + encoder_categories_gend[1:]
#print(col_names)
CIP_data_encoded_gend.columns = col_names
#
print(CIP_data_encoded_gend.head())


######### Try "get dummies" method

#X = pd.get_dummies(CIP_data[['GENDERSP']])
#print(X.head())

   Unnamed: 0 CLINIC  YEAR GENDERSP  Susceptible  MSMW  MSW  Oth/Unk/Missing
0           0    ALB  2000      MSW            1   0.0  1.0              0.0
1           0    ALB  2000      MSW            1   0.0  1.0              0.0
2           0    ALB  2000      MSW            1   0.0  1.0              0.0
3           0    ALB  2000      MSW            1   0.0  1.0              0.0
4           0    ALB  2000      MSW            1   0.0  1.0              0.0


In [5]:
###Now need to fix it so that things are coded by region 
### Coding

west = ['POR', 'PHX', 'HON', 'SDG', 'SFO', 'ANC', 'SEA', 'DEN', 'LVG', 'ORA', 'LBC', 'SLC', 'LAX']
southwest = ['OKC','MIN', 'ALB', 'DAL']
midwest = ['KCY','CHI', 'PON', 'CIN', 'JAC', 'IND', 'STL','DTR', 'MIL', 'COL', 'CLE']
southeast = ['GRB', 'NOR','WDC','MIA', 'BHM','FBG','ATL', 'RIC']
northeast = ['BUF','BOS', 'CAM', 'NYC', 'BAL', 'PHI']

### One-hot encoding 
CIP_data_encoded_gend['REGION'] = CIP_data_encoded_gend['CLINIC'].apply(lambda x: 
    'West' if (x in west) else (
        'Southwest' if (x in southwest) else(
            'Midwest' if (x in midwest) else(
                'Southeast' if (x in southeast) else(
                    'Northeast' if (x in northeast) else 'Other'))))) 

#print(CIP_data_encoded_gend.head())


encoder_region = OneHotEncoder(drop='first')
encoder_categories_region  = encoder_region.fit(CIP_data_encoded_gend[['REGION']]).categories_
encoder_categories_region  = encoder_categories_region[0].tolist()
encoder_df_region = pd.DataFrame(encoder_region.fit_transform(CIP_data_encoded_gend[['REGION']]).toarray())
CIP_data_encoded_gend_region = CIP_data_encoded_gend.join(encoder_df_region)
col_names = list(CIP_data_encoded_gend.columns) + encoder_categories_region[1:]
#print(col_names)
CIP_data_encoded_gend_region.columns = col_names
print(CIP_data_encoded_gend_region.head())


   Unnamed: 0 CLINIC  YEAR GENDERSP  Susceptible  MSMW  MSW  Oth/Unk/Missing  \
0           0    ALB  2000      MSW            1   0.0  1.0              0.0   
1           0    ALB  2000      MSW            1   0.0  1.0              0.0   
2           0    ALB  2000      MSW            1   0.0  1.0              0.0   
3           0    ALB  2000      MSW            1   0.0  1.0              0.0   
4           0    ALB  2000      MSW            1   0.0  1.0              0.0   

      REGION  Northeast  Southeast  Southwest  West  
0  Southwest        0.0        0.0        1.0   0.0  
1  Southwest        0.0        0.0        1.0   0.0  
2  Southwest        0.0        0.0        1.0   0.0  
3  Southwest        0.0        0.0        1.0   0.0  
4  Southwest        0.0        0.0        1.0   0.0  


In [17]:
##### Now try the previous year's prevalence 
##### First, region 
##### NB here the "Susceptible" means "Susceptible to CIPRO", so 1-susceptible is resistance 
suscep_by_region = (CIP_data_encoded_gend_region.groupby(by = ["REGION"])["Susceptible"].sum())
print(suscep_by_region)

regions = CIP_data_encoded_gend_region["REGION"].unique()
years = CIP_data_encoded_gend_region["YEARS"].unique()
prevalence_by_region = {}
for region in regions:
    print(region)
    print(suscep_by_region[region])
    prevalence_by_region[region] = suscep_by_region[region]/len(CIP_data_encoded_gend_region["REGION"] == region)
#/CIP_data_encoded_gend_region.groupby(by = ["REGION"])["Susceptible"].size()

print(prevalence_by_region)

REGION
Midwest      23475
Northeast    11155
Southeast    17545
Southwest    12839
West         31175
Name: Susceptible, dtype: int64
Southwest
12839
West
31175
Southeast
17545
Northeast
11155
Midwest
23475
{'Southwest': 0.11413763368211438, 'West': 0.27714313654022243, 'Southeast': 0.15597357916914845, 'Northeast': 0.09916701485504992, 'Midwest': 0.2086907820459253}


In [58]:
##Now try by year 

regions = CIP_data_encoded_gend_region["REGION"].unique()
years = CIP_data_encoded_gend_region["YEAR"].unique()
for year in years:
    CIP_data_year = CIP_data_encoded_gend_region.loc[CIP_data_encoded_gend_region['YEAR'] == year]
    suscep_by_region = (CIP_data_year.groupby(by = ["REGION"])["Susceptible"].sum())
    
    prevalence_by_region = {}
    for region in regions:
            prevalence_by_region[region] = 1 - suscep_by_region[region]/len(CIP_data_year[CIP_data_year.REGION == region])
            #CIP_data_encoded_gend_region["PREV_REGION"] = np.where((CIP_data_encoded_gend_region["YEAR"] == year) & (CIP_data_encoded_gend_region["REGION"] == region), prevalence_by_region[region])
            if year == 2000:
                CIP_data_encoded_gend_region.loc[(CIP_data_encoded_gend_region["YEAR"] == 2000) & (CIP_data_encoded_gend_region["REGION"] == region), "PREV_REGION"] = 0
            else:
                CIP_data_encoded_gend_region.loc[(CIP_data_encoded_gend_region["YEAR"] == year + 1) & (CIP_data_encoded_gend_region["REGION"] == region), "PREV_REGION"] = prevalence_by_region[region]


print(CIP_data_encoded_gend_region["PREV_REGION"])

0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
            ...   
112482    0.282092
112483    0.282092
112484    0.282092
112485    0.282092
112486    0.282092
Name: PREV_REGION, Length: 112487, dtype: float64


In [56]:
### checking against the 2019 data 
CIP_data_2019 = CIP_data_encoded_gend_region.loc[CIP_data_encoded_gend_region['YEAR'] == 2019]
#print(1 - CIP_data_2019['Susceptible'].sum()/len(CIP_data_2019['Susceptible'])) # 1 - susceptible to cipro is resistant... correct

#print(CIP_data_2019.groupby(by = ["REGION"]))
print(CIP_data_2019["REGION"].unique())
#print(CIP_data_2019.loc[CIP_data_2019["REGION"] == "West", "Susceptible"].sum())

print(len(CIP_data_2019[CIP_data_2019.REGION == "West"]))

suscep_by_region = (CIP_data_2019.groupby(by = ["REGION"])["Susceptible"].sum())

regions = CIP_data_2019["REGION"].unique()

prevalence_by_region = {}
for region in regions:
    print(region)
    print(suscep_by_region[region])
    prevalence_by_region[region] = 1 - suscep_by_region[region]/len(CIP_data_2019[CIP_data_2019.REGION == region])
#/CIP_data_encoded_gend_region.groupby(by = ["REGION"])["Susceptible"].size()

print(prevalence_by_region)
CIP_data_2019.tail()

['Southwest' 'West' 'Northeast' 'Southeast' 'Midwest']
1974
Southwest
350
West
1184
Northeast
410
Southeast
397
Midwest
1194
{'Southwest': 0.3027888446215139, 'West': 0.4002026342451874, 'Northeast': 0.4576719576719577, 'Southeast': 0.3190394511149228, 'Midwest': 0.2802893309222423}


,Unnamed: 0,CLINIC,YEAR,GENDERSP,Susceptible,MSMW,MSW,Oth/Unk/Missing,REGION,Northeast,Southeast,Southwest,West,PREV_REGION
112482,1968,WDC,2019,MSMW,0,1.0,0.0,0.0,Southeast,0.0,1.0,0.0,0.0,0.319039
112483,1968,WDC,2019,MSMW,0,1.0,0.0,0.0,Southeast,0.0,1.0,0.0,0.0,0.319039
112484,1968,WDC,2019,MSMW,0,1.0,0.0,0.0,Southeast,0.0,1.0,0.0,0.0,0.319039
112485,1968,WDC,2019,MSMW,0,1.0,0.0,0.0,Southeast,0.0,1.0,0.0,0.0,0.319039
112486,1969,WDC,2019,Oth/Unk/Missing,0,0.0,0.0,1.0,Southeast,0.0,1.0,0.0,0.0,0.319039
